<p align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Apache_Spark_logo.svg/2560px-Apache_Spark_logo.svg.png" width="20%"/>
</p>

# PySpark
---
```Bash
Author: Witchakorn Wanasanwongkot
```

## Spark
Spark is a distributed cluster computing framework for big data processing. It prevides a parallel abstraction for manipulating and processing the data.

In [9]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [4]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [32]:
# Overview
df = spark.read.csv('data/superstore_dataset2011-2015.csv', header=True, inferSchema=True)
print((df.count(), len(df.columns)))
df.limit(3).toPandas()

(51290, 22)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,42433,AG-2011-2040,01-01-11,06-01-11,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.3,2,0,106.140
1,22253,IN-2011-47883,01-01-11,08-01-11,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036
2,48883,HU-2011-1220,01-01-11,05-01-11,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0,29.640


In [10]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Market: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping Cost: double (nullable = true)
 |-- Order Priority: string (nullable = true)



### Transform schema

In [34]:
cols = df.columns

types = [
    IntegerType(),
    StringType(),
    DateType(),
    DateType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    StringType(),
    DoubleType(),
    IntegerType(),
    DoubleType(),
    DoubleType(),
]

schema_list = []
for i in range(len(cols)):
    schema_list.append(StructField(cols[i], types[i]))

schema = StructType(schema_list)
print(schema_list[0], schema, sep='\n\n')

StructField('Row ID', IntegerType(), True)

StructType([StructField('Row ID', IntegerType(), True), StructField('Order ID', StringType(), True), StructField('Order Date', DateType(), True), StructField('Ship Date', DateType(), True), StructField('Ship Mode', StringType(), True), StructField('Customer ID', StringType(), True), StructField('Customer Name', StringType(), True), StructField('Segment', StringType(), True), StructField('City', StringType(), True), StructField('State', StringType(), True), StructField('Country', StringType(), True), StructField('Postal Code', StringType(), True), StructField('Market', StringType(), True), StructField('Region', StringType(), True), StructField('Product ID', StringType(), True), StructField('Category', StringType(), True), StructField('Sub-Category', StringType(), True), StructField('Product Name', StringType(), True), StructField('Sales', DoubleType(), True), StructField('Quantity', IntegerType(), True), StructField('Discount', DoubleType(), T

In [40]:
# Apply a new schema to the DataFrame and change the date format
df = spark.read.csv('data/superstore_dataset2011-2015.csv', header=True, schema=schema, dateFormat='dd-MM-yy')
print((df.count(), len(df.columns)))
df.limit(3).toPandas()

(51290, 22)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,42433,AG-2011-2040,2011-01-01,2011-01-06,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,None,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140
1,22253,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,None,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036
2,48883,HU-2011-1220,2011-01-01,2011-01-05,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,None,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640


In [41]:
# Drop the unnecessary column
df = df.drop('Row ID')

## Selection & Filtering

In [42]:
# Select the specific column
df.select(['Order ID', 'Sales', 'Profit']).limit(10).toPandas()

,Order ID,Sales,Profit
0,AG-2011-2040,408.300,106.1400
1,IN-2011-47883,120.366,36.0360
2,HU-2011-1220,66.120,29.6400
3,IT-2011-3647632,44.865,-26.0550
4,IN-2011-47883,113.670,37.7700
5,IN-2011-47883,55.242,15.3420
6,IN-2011-30733,285.780,71.4000
7,CA-2011-115161,290.666,3.4196
8,AO-2011-1390,206.400,92.8800
9,ID-2011-56493,162.720,68.3100


In [43]:
# Filter the data to display only entries with profits over 500
df.filter(df.Profit > 500).limit(5).toPandas()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2011-131926,2011-06-01,2011-06-06,Second Class,DW-13480,Dianna Wilson,Home Office,Lakeville,Minnesota,United States,55044,US,Central,FUR-CH-10004063,Furniture,Chairs,Global Deluxe High-Back Manager's Chair,2001.86,7,0.0,580.5394
1,IT-2011-4243443,2011-11-01,2011-11-06,Second Class,DF-13135,David Flashing,Consumer,Slough,England,United Kingdom,None,EU,North,TEC-CO-10000013,Technology,Copiers,"Brother Fax Machine, Laser",1592.70,5,0.0,652.9500
2,CA-2011-120474,2011-12-01,2011-12-03,First Class,RP-19390,Resi P�lking,Consumer,Madison,Wisconsin,United States,53711,US,Central,FUR-CH-10001854,Furniture,Chairs,Office Star - Professional Matrix Back Chair w...,2807.84,8,0.0,673.8816
3,ES-2011-5125239,2011-12-01,2011-12-06,Standard Class,MG-17875,Michael Grace,Home Office,Genk,Limburg,Belgium,None,EU,Central,TEC-AC-10001258,Technology,Accessories,"Belkin Router, Erganomic",1795.92,7,0.0,843.9900
4,ES-2011-3614277,2011-12-01,2011-12-03,Second Class,SL-20155,Sara Luxemburg,Home Office,Valencia,Valenciana,Spain,None,EU,South,TEC-CO-10004042,Technology,Copiers,"Brother Wireless Fax, Laser",1894.05,5,0.0,700.6500


In [44]:
# Select & Filter the data to display only entries with profits less than 0 and region is 'West'
df.select(['Region', 'Order ID', 'Sales', 'Profit']) \
    .filter(df.Profit < 0).filter(df.Region == 'West') \
    .limit(10) \
    .toPandas()

,Region,Order ID,Sales,Profit
0,West,CA-2011-148915,443.920,-94.3330
1,West,CA-2011-108189,15.360,-3.2640
2,West,CA-2011-108189,7.160,-0.0895
3,West,CA-2011-105648,626.352,-23.4882
4,West,CA-2011-133690,218.750,-161.8750
5,West,CA-2011-125829,573.728,-64.5444
6,West,US-2011-112872,275.490,-170.8038
7,West,US-2011-163797,49.792,-11.8256
8,West,CA-2011-100090,502.488,-87.9354
9,West,CA-2011-151708,95.976,-10.7973


### SQLs

In [45]:
# Create TempView
df.createOrReplaceTempView('superstore')

In [47]:
# Query the data to display only entries with profits less than 0 and region is 'West'
spark.sql("""
    SELECT 
        Region,
        `Order ID`,
        Sales,
        Profit
    FROM superstore
    WHERE Profit < 0 AND Region = 'West'
""").limit(10).toPandas()

,Region,Order ID,Sales,Profit
0,West,CA-2011-148915,443.920,-94.3330
1,West,CA-2011-108189,15.360,-3.2640
2,West,CA-2011-108189,7.160,-0.0895
3,West,CA-2011-105648,626.352,-23.4882
4,West,CA-2011-133690,218.750,-161.8750
5,West,CA-2011-125829,573.728,-64.5444
6,West,US-2011-112872,275.490,-170.8038
7,West,US-2011-163797,49.792,-11.8256
8,West,CA-2011-100090,502.488,-87.9354
9,West,CA-2011-151708,95.976,-10.7973


## Descriptive Function
Descriptive function provides short summary in single dimension of dataset.

In [48]:
df.describe(['Sales', 'Profit']).toPandas()

,summary,Sales,Profit
0,count,50990,51290
1,mean,247.4554011743472,28.597540880873467
2,stddev,488.71690163686173,174.3747508696452
3,min,0.444,-6599.978
4,max,22638.48,8399.976


In [49]:
# Count the number of distinct values of `Product Name` and display them
print(df.select('Product Name').distinct().count())
df.select('Product Name').distinct().limit(10).toPandas()

3785


,Product Name
0,"Chromcraft Computer Table, Fully Assembled"
1,"SanDisk Router, Erganomic"
2,"Avery 3-Hole Punch, Recycled"
3,Wilson Jones 14 Line Acrylic Coated Pressboard...
4,"SanDisk Keyboard, Programmable"
5,"Dania 3-Shelf Cabinet, Mobile"
6,Aastra 6757i CT Wireless VoIP phone
7,"Global Armless Task Chair, Royal Blue"
8,"Samsung Headset, VoIP"
9,"KitchenAid Refrigerator, Black"


In [51]:
# Count the number for each `Product Name`
df.groupBy('Product Name').count().sort('count', ascending=False).limit(10).toPandas()

,Product Name,count
0,Staples,227
1,"Cardinal Index Tab, Clear",92
2,"Eldon File Cart, Single Width",90
3,"Rogers File Cart, Single Width",84
4,"Ibico Index Tab, Clear",83
5,"Sanford Pencil Sharpener, Water Color",80
6,"Smead File Cart, Single Width",77
7,"Stanley Pencil Sharpener, Water Color",75
8,"Acco Index Tab, Clear",75
9,"Avery Index Tab, Clear",74


## Enrichment
Enrichment is the process of which emphasize the detail of the data. Sometimes involved with increase in data dimension.

In [52]:
# Creat a new column based on an existing column (`Shipping goal date` = `Order Date` + 3)
df = df.withColumn('Shipping goal date', date_add(col('Order Date'), 3))
df.select(['Order ID', 'Order Date', 'Shipping goal date', 'Ship Date']).limit(10).toPandas()

,Order ID,Order Date,Shipping goal date,Ship Date
0,AG-2011-2040,2011-01-01,2011-01-04,2011-01-06
1,IN-2011-47883,2011-01-01,2011-01-04,2011-01-08
2,HU-2011-1220,2011-01-01,2011-01-04,2011-01-05
3,IT-2011-3647632,2011-01-01,2011-01-04,2011-01-05
4,IN-2011-47883,2011-01-01,2011-01-04,2011-01-08
5,IN-2011-47883,2011-01-01,2011-01-04,2011-01-08
6,IN-2011-30733,2011-02-01,2011-02-04,2011-02-03
7,CA-2011-115161,2011-02-01,2011-02-04,2011-02-03
8,AO-2011-1390,2011-02-01,2011-02-04,2011-02-04
9,ID-2011-56493,2011-02-01,2011-02-04,2011-02-03


In [53]:
# Determine if the order was shipped late, compare the delivery date (`Ship date`) with the estimated delivery date (`Shipping goal date`)
df = df.withColumn('Is Shipped Late', when(datediff(col('Shipping goal date'), col('Ship Date')) >= 0, 'No').otherwise('Yes'))
df.select(['Order ID', 'Order Date', 'Shipping goal date', 'Ship Date', 'Is Shipped Late']).limit(10).toPandas()

,Order ID,Order Date,Shipping goal date,Ship Date,Is Shipped Late
0,AG-2011-2040,2011-01-01,2011-01-04,2011-01-06,Yes
1,IN-2011-47883,2011-01-01,2011-01-04,2011-01-08,Yes
2,HU-2011-1220,2011-01-01,2011-01-04,2011-01-05,Yes
3,IT-2011-3647632,2011-01-01,2011-01-04,2011-01-05,Yes
4,IN-2011-47883,2011-01-01,2011-01-04,2011-01-08,Yes
5,IN-2011-47883,2011-01-01,2011-01-04,2011-01-08,Yes
6,IN-2011-30733,2011-02-01,2011-02-04,2011-02-03,No
7,CA-2011-115161,2011-02-01,2011-02-04,2011-02-03,No
8,AO-2011-1390,2011-02-01,2011-02-04,2011-02-04,No
9,ID-2011-56493,2011-02-01,2011-02-04,2011-02-03,No


In [54]:
# Extract the year, month, and day values from the date column
df = df.withColumn('Year', year('Order Date')).withColumn('Month', month('Order Date')).withColumn('Day', dayofmonth('Order Date'))
df.select(['Order ID', 'Year', 'Month', 'Day']).limit(10).toPandas()

,Order ID,Year,Month,Day
0,AG-2011-2040,2011,1,1
1,IN-2011-47883,2011,1,1
2,HU-2011-1220,2011,1,1
3,IT-2011-3647632,2011,1,1
4,IN-2011-47883,2011,1,1
5,IN-2011-47883,2011,1,1
6,IN-2011-30733,2011,2,1
7,CA-2011-115161,2011,2,1
8,AO-2011-1390,2011,2,1
9,ID-2011-56493,2011,2,1


## Aggregation
Data aggregation is a process in which transform the information and display in summary.

In [56]:
# Identify the top spender, list the name of the individual who has spent the most
df.groupBy('Customer ID') \
    .agg(first('Customer Name'), sum(df['Sales'])) \
    .sort(col('sum(Sales)').desc()) \
    .limit(10) \
    .toPandas()

,Customer ID,first(Customer Name),sum(Sales)
0,TA-21385,Tom Ashbrook,35668.12080
1,GT-14710,Greg Tran,34471.89028
2,TC-20980,Tamara Chand,34183.89900
3,SM-20320,Sean Miller,31125.29496
4,BW-11110,Bart Watters,30613.61650
5,HL-15040,Hunter Lopez,29664.23058
6,SE-20110,Sanjit Engle,29532.62502
7,PS-19045,Penelope Sewall,29252.31940
8,RB-19360,Raymond Buch,29197.63460
9,ZC-21910,Zuschuss Carroll,28468.24726


In [59]:
# Divide each customer's spending into categories
sales_per_customer = df.groupBy(['Customer ID', 'Customer name']) \
                        .pivot('Category') \
                        .agg(sum('Sales')) \
                        .fillna(0)

sales_per_customer = sales_per_customer.withColumn('Total', expr('Furniture + `Office Supplies` + Technology'))
sales_per_customer.sort(col('Total').desc()).limit(10).toPandas()

,Customer ID,Customer name,Furniture,Office Supplies,Technology,Total
0,TA-21385,Tom Ashbrook,3350.6235,12024.4565,20293.04080,35668.12080
1,GT-14710,Greg Tran,15690.0470,7686.7940,11095.04928,34471.89028
2,TC-20980,Tamara Chand,7803.3390,3422.2430,22958.31700,34183.89900
3,SM-20320,Sean Miller,1669.7460,3879.1616,25576.38736,31125.29496
4,BW-11110,Bart Watters,12023.8106,7191.1989,11398.60700,30613.61650
5,HL-15040,Hunter Lopez,5517.0180,3191.4659,20955.74668,29664.23058
6,SE-20110,Sanjit Engle,6136.7955,4773.9840,18621.84552,29532.62502
7,PS-19045,Penelope Sewall,10674.5736,8656.3581,9921.38770,29252.31940
8,RB-19360,Raymond Buch,4720.4930,6018.2140,18458.92760,29197.63460
9,ZC-21910,Zuschuss Carroll,6122.1111,9235.3917,13110.74446,28468.24726


In [60]:
# Summarize total sales by sub-category
df.groupBy('Sub-Category') \
    .agg(sum('Sales')) \
    .toPandas()

,Sub-Category,sum(Sales)
0,Envelopes,1.697674e+05
1,Art,3.720920e+05
2,Chairs,1.501682e+06
3,Furnishings,3.766253e+05
4,Supplies,2.423532e+05
5,Fasteners,8.322669e+04
6,Binders,4.584045e+05
7,Bookcases,1.466572e+06
8,Labels,7.340403e+04
9,Paper,2.411686e+05


In [62]:
# Divide sales of each sub-category into sales from each customer segment
df.groupBy('Sub-Category') \
    .pivot('Segment') \
    .agg(sum('Sales')) \
    .fillna(0) \
    .toPandas()

,Sub-Category,Consumer,Corporate,Home Office
0,Envelopes,88059.14810,51230.04480,30478.19670
1,Art,200870.61440,103571.37720,67649.97430
2,Chairs,778362.53010,448519.51280,274799.72130
3,Furnishings,198474.54700,109992.36600,68158.40890
4,Supplies,122827.23310,81354.10110,38171.81840
5,Fasteners,42063.24920,26136.43170,15027.01100
6,Binders,252311.13460,126582.03670,79511.31840
7,Bookcases,765111.14480,457327.16840,244133.92860
8,Labels,38698.03520,21192.73370,13513.26110
9,Paper,118109.11510,71135.05530,51924.46100


### Window function

In [70]:
# Calculate the contribution of each sub-category to the total sales of each customer segment, using a table-down approach
window = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df.groupBy('Sub-Category') \
    .pivot('Segment') \
    .agg(sum('Sales')) \
    .fillna(0) \
    .withColumn('Consumer Total', sum(col('Consumer')).over(window)) \
    .withColumn('Percent Consumer', col('Consumer')*100/col('Consumer Total')) \
    .drop(col('Consumer Total')) \
    .withColumn('Corporate Total', sum(col('Corporate')).over(window)) \
    .withColumn('Percent Corporate', col('Corporate')*100/col('Corporate Total')) \
    .drop(col('Corporate Total')) \
    .withColumn('Home Office Total', sum(col('Home Office')).over(window)) \
    .withColumn('Percent Home Office', col('Home Office')*100/col('Home Office Total')) \
    .drop(col('Home Office Total')) \
    .toPandas()

,Sub-Category,Consumer,Corporate,Home Office,Percent Consumer,Percent Corporate,Percent Home Office
0,Envelopes,88059.14810,51230.04480,30478.19670,1.355441,1.342804,1.321759
1,Art,200870.61440,103571.37720,67649.97430,3.091880,2.714735,2.933802
2,Chairs,778362.53010,448519.51280,274799.72130,11.980864,11.756257,11.917342
3,Furnishings,198474.54700,109992.36600,68158.40890,3.054999,2.883037,2.955851
4,Supplies,122827.23310,81354.10110,38171.81840,1.890605,2.132393,1.655411
5,Fasteners,42063.24920,26136.43170,15027.01100,0.647454,0.685069,0.651682
6,Binders,252311.13460,126582.03670,79511.31840,3.883673,3.317873,3.448197
7,Bookcases,765111.14480,457327.16840,244133.92860,11.776894,11.987117,10.587447
8,Labels,38698.03520,21192.73370,13513.26110,0.595655,0.555488,0.586035
9,Paper,118109.11510,71135.05530,51924.46100,1.817982,1.864539,2.251827


In [71]:
# Calculate the contribution of each customer segment to the total sales of each sub-category, using a table-across approach
df.groupBy('Sub-Category') \
    .pivot('Segment') \
    .agg(sum('Sales')) \
    .fillna(0) \
    .withColumn('Total', expr('Consumer + Corporate + `Home Office`')) \
    .withColumn('Percent Consumer', col('Consumer')*100/col('Total')) \
    .withColumn('Percent Corporate', col('Corporate')*100/col('Total')) \
    .withColumn('Percent Home Office', col('Home Office')*100/col('Total')) \
    .withColumn('Percent Total', expr('`Percent Consumer` + `Percent Corporate` + `Percent Home Office`')) \
    .toPandas()

,Sub-Category,Consumer,Corporate,Home Office,Total,Percent Consumer,Percent Corporate,Percent Home Office,Percent Total
0,Envelopes,88059.14810,51230.04480,30478.19670,1.697674e+05,51.870473,30.176611,17.952916,100.0
1,Art,200870.61440,103571.37720,67649.97430,3.720920e+05,53.984131,27.834887,18.180982,100.0
2,Chairs,778362.53010,448519.51280,274799.72130,1.501682e+06,51.832722,29.867814,18.299464,100.0
3,Furnishings,198474.54700,109992.36600,68158.40890,3.766253e+05,52.698142,29.204719,18.097139,100.0
4,Supplies,122827.23310,81354.10110,38171.81840,2.423532e+05,50.681096,33.568410,15.750494,100.0
5,Fasteners,42063.24920,26136.43170,15027.01100,8.322669e+04,50.540576,31.403906,18.055519,100.0
6,Binders,252311.13460,126582.03670,79511.31840,4.584045e+05,55.041157,27.613612,17.345231,100.0
7,Bookcases,765111.14480,457327.16840,244133.92860,1.466572e+06,52.170028,31.183405,16.646567,100.0
8,Labels,38698.03520,21192.73370,13513.26110,7.340403e+04,52.719224,28.871349,18.409427,100.0
9,Paper,118109.11510,71135.05530,51924.46100,2.411686e+05,48.973664,29.495982,21.530354,100.0
